Confidence <br/>
![](http://www.sciweavers.org/upload/Tex2Img_1651907171/render.png)

<br/>
Assume:
Data is normal

# Imports & loading

In [1]:
from statistics import *

import numpy as np
import scipy
import pandas as pd
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pio.templates.default = "plotly_dark"

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 100)
pd.set_option("plotting.backend", "plotly")

def normalize(v):
    norm=np.linalg.norm(v)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm

In [2]:
df_original = pd.read_csv("exp_survey.csv", encoding="utf8")

# Cleaning

In [3]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 76 columns):
 #   Column                                                                                              Non-Null Count  Dtype  
---  ------                                                                                              --------------  -----  
 0   Timestamp                                                                                           30 non-null     object 
 1   الجنس                                                                                               30 non-null     object 
 2   التخصص                                                                                              30 non-null     object 
 3   الدفعة                                                                                              30 non-null     object 
 4   أعلى درجة قدرات لك (مثلا: "83")                                                                     30 non-null     in

In [4]:
df = pd.read_csv("exp_survey.csv", encoding="utf8")

# Manual editing, yikes!
df.at[9, "قدر عدد الجهات اللتي قدمت عليها"] = "15"
df.at[20, "قدر عدد الجهات اللتي قدمت عليها"] = "15"
df.at[20, "قدر عدد الجهات اللتي ردت عليك"] = "6"
df.at[20, "قدر عدد الجهات اللتي ردت عليك بالرفض"] = "4"
df.at[25, "مساحة تعليقات حرة.3"] = np.nan
df.at[26, "ماهي المجالات المهتم فيها"] = np.nan
df.at[22, "ماهي المجالات المهتم فيها"] = np.nan
df.at[28, "ماهي المجالات المهتم فيها"] = "Frontend, Backend, Mobile, Data analysis, Blockchain, ML"

# Macro filtering
df = (df
    .drop("Timestamp", axis=1)
    .dropna(how="all", axis=1) # Drops empty columns
)

# Columns breakdown 
comments_cols = [col for col in df.columns if "مساحة" in col or "نصائح" in col]
categorical_cols = [col for col in df.columns if "هل" in col or ("كيف" in col and col != "كيف طبيعة التطوع")]
categorical_cols.extend(["الجنس", "التخصص", "الدفعة", "بعد أي مستوى كان التدريب", "كم كان الأجر بالشهر (بالآف)"])
checkbox_cols = [
    "ماهي المجالات المهتم فيها",
    "كيف طبيعة التطوع",
    "في أي المجالات التالية كانت مشاريعك",
    "ما نوع الاسئلة في المقابلة",
    "في أي المجالات التالية كانت تدريبك"
]
number_cols = set(df.columns).difference(set((*checkbox_cols, *categorical_cols, *comments_cols)))


for col in categorical_cols:
    df[col] = df[col].astype("category")


# Expand checkbox cols to multiple boolean cols
for col in checkbox_cols:
    categories = set(val.strip() for val in (",".join(df[col].dropna().values)).split(","))
    for category in categories:
        new_col = f"{col} [{category}]"
        df[new_col] = df[col].apply(lambda x: np.NaN if not isinstance(x, str) else category in x)
    df = df.drop(col, axis=1)


for col in number_cols:
    df[col] = df[col].astype("float32")


renamed_cols = [
    "الجنس",
    "التخصص",
    "الدفعة",
    "القدرات",
    "التحصيلي",
    "المعدل",
    "عدد الصيفيات",
    "عدد الحذف",
    "عدد الحمل",
    "هل غيرت تخصصك",
    "هل تخصصك رغبتك الأولى",
    "هل كان عندك معرفه جيدة بتخصصك",
    "Interest Frontend",
    "Interest Backend",
    "Interest Mobile",
    "Interest Data analysis",
    "Interest Databases",
    "Interest Cyber Security",
    "Interest Academia",
    "Experience Frontend",
    "Experience Backend",
    "Experience Mobile",
    "Experience Data analysis",
    "Experience Databases",
    "Experience Cyber Security",
    "Experience Academia",
    'قدّر ميولك المهني ضد ميولك الأكاديمي',
    'تطوعت سابقا',
    "كيف طبيعة التطوع"
    'قدّر فائدة التطوع على مسيرتك المهنية', 
    'تنصح بالتطوع',
    'مساحة تعليقات حرة.1',
    'عمل سابقا',
    'كيف طبيعة العمل', 
    'كم كان الأجر بالشهر (بالآف)',
    'قدّر فائدة العمل على مسيرتك المهنية', 
    'هل تنصح غيرك بعمل مماثل',
    'هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب',
    'قدّر فائدة مشاريعك ككل بالنسبة للوقت المصروف عليها حسب مسيرتك المهنية',
    'هل كان لمشاريعك الخارجية أثر مباشر على فرصك المهنية (تدريب، توظيف، الخ)',
    'هل تنصح غيرك بالمشاريع الخارجية',
    'هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)',
    'هل لازلت تبحث عن تدريب', 
    'قدر عدد الجهات اللتي قدمت عليها',
    'قدر عدد الجهات اللتي ردت عليك',
    'قدر عدد الجهات اللتي ردت عليك بالرفض',
    'قدر عدد الجهات اللتي ردت عليك بالقبول',
    'هل قبلت في أي جهة (قبول أولي على الأقل)',
    'هل لديك سيرة ذاتية (CV)',
    'قدر فائدة السيرة الذاتية',
    'نصائح كتابة سيرة ذاتية',
    'هل لديك حساب في Linkedin',
    'قدر فائدة حساب لنكدان',
    'هل لديك حساب في Github',
    'قدر فائدة حساب قتهب',
    'مساحة تعليقات حرة.4', 
    'هل أجريت مقابلة تدريب',
    'ما مدى صعوبة المقابلة بالنسبة لمهاراتك',
    'هل المقابلة تطلبت منك البرمجة', 
    'نصائح قبل المقابلة ',
    'تدرب سابقا', 
    'هل التدريب كان في فصل صيفي',
    'بعد أي مستوى كان التدريب',
    'ما مدى صعوبة التدريب بالنسبة لمهاراتك',
    'مدة التدريب',
    'تدريب بسبب علاقات',
    'وظيفة بعد التدريب', 
    'مساحة تعليقات حرة.5',
    'Interests Game dev',
    'Interests Cyber Security',
    'Interests Mobile',
    'Interests UX Design',
    'Interests Software testing',
    'Interests Product management',
    'Interests Frontend',
    'Interests Graphics',
    'Interests Databases',
    'Interests Data analysis',
    'Interests Data engineering',
    'Interests Blockchain',
    'Interests ML',
    'Interests business analysis',
    'Interests Backend',
    'Interests Academia',
    'تطوع نادي', 
    'تطوع خارجي',
    'Projects Game dev',
    'Projects Cyber Security',
    'Projects Algorithm design',
    'Projects UX Design',
    'Projects game development',
    'Projects Frontend',
    'Projects Mobile app',
    'Projects Blockchain + ML',
    'Projects Databases',
    'Projects Data analysis',
    'Projects Backend',
    'Projects Academia',
    'اسئلة تخصصات دقيقة',
    'اسئلة حاسب عامة',
    'اسئلة موارد بشرية',
    'Internship Cyber Security',
    'Internship Quality Assurance',
    'Internship Product management',
    'Internship Databases',
    'Internship Data analysis',
    'Internship ML',
]
name_map = {old:new for old, new in zip(df.columns[:len(renamed_cols)], renamed_cols)}
comments_cols = [name_map.get(col, col) for col in comments_cols]
categorical_cols = [name_map.get(col, col) for col in categorical_cols]
checkbox_cols = [name_map.get(col, col) for col in checkbox_cols]
number_cols = [name_map.get(col, col) for col in number_cols]
df.columns = [name_map.get(col, col) for col in df.columns]

# Sort columns
df = df.reindex(sorted(df.columns), axis=1)

#### Define new cols

In [5]:
def gpa_map(val):
    if val >= 4.5:
        return "ممتاز"
    elif val >= 3.75:
        return "جيد جدا"
    elif val >= 2.75:
        return "جيد"
    elif val >= 2:
        return "مقبول"
    elif val < 2:
        return "راسب"
df["التصنيف"] = df["المعدل"].apply(gpa_map)


df["معدل القبول"] = df["قدر عدد الجهات اللتي ردت عليك بالقبول"] / df["قدر عدد الجهات اللتي قدمت عليها"]

# Meta analysis

#### Survey structure
###### What are the different sections on a survey

#### Counts analysis
###### How many people particiapted in each section

In [56]:
section_questions = [
    "تطوعت سابقا",
    "عمل سابقا",
    "هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب",
    "هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)",
    "هل لازلت تبحث عن تدريب",
    "هل قبلت في أي جهة (قبول أولي على الأقل)",
    "هل أجريت مقابلة تدريب",
    "تدرب سابقا",
]

In [57]:
temp = (df[section_questions] == "نعم").sum()
question2num_answers = dict(zip(temp.index, temp.values))
question2num_answers

{'تطوعت سابقا': 13,
 'عمل سابقا': 7,
 'هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب': 17,
 'هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)': 18,
 'هل لازلت تبحث عن تدريب': 13,
 'هل قبلت في أي جهة (قبول أولي على الأقل)': 10,
 'هل أجريت مقابلة تدريب': 9,
 'تدرب سابقا': 4}

In [98]:
sections = dict(
    sections=[
        "كل الأقسام", 
        "قسم النشاطات اللاصفية", 
        "فسم التطوع", 
        "قسم العمل", 
        "قسم المشاريع", 
        "قسم التدريب", 
        "قسم  المقبولين (قبول أولي على الأقل) في التدريب", 
        "قسم من أجروا المقابلة",
        "قسم المتدربين سابقا",
        ],
    parents=[
        "",
        "كل الأقسام",
        "قسم النشاطات اللاصفية",
        "قسم النشاطات اللاصفية",
        "قسم النشاطات اللاصفية",
        "كل الأقسام",
        "قسم التدريب",
        "قسم  المقبولين (قبول أولي على الأقل) في التدريب",
        "قسم التدريب",
        ],
    participants=[
        len(df),
        sum(df[["تطوعت سابقا", "عمل سابقا", "هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب"]].apply(lambda row: 1 if "نعم" in row.values else 0, axis=1)),
        question2num_answers["تطوعت سابقا"],
        question2num_answers["عمل سابقا"],
        question2num_answers["هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب"],
        question2num_answers["هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)"],
        question2num_answers["هل قبلت في أي جهة (قبول أولي على الأقل)"],
        question2num_answers["هل أجريت مقابلة تدريب"],
        question2num_answers["تدرب سابقا"],
    ]
)

fig =px.icicle(
    sections,
    names="sections",
    parents="parents",
    values="participants",
    color='participants'
)
fig.update_traces(
            tiling = dict(
            orientation='v',
            flip='x'
        )
)
fig.update_layout(
    height=1000,
    width=2200,
    font=dict(
        size=25,
    )

)
fig.show()


# Analysis

In [7]:
# Utility for interests and experiences
interests_cols = [col for col in df.columns if "Interest " in col]
experiences_cols = [col for col in df.columns if "Experience" in col]
field_names = [col[col.find("[")+1:col.find("]")] for col in interests_cols]
applying_cols = ["قدر عدد الجهات اللتي قدمت عليها", "قدر عدد الجهات اللتي ردت عليك", "قدر عدد الجهات اللتي ردت عليك بالرفض", "قدر عدد الجهات اللتي ردت عليك بالقبول", "معدل القبول"]
grade_cols = ["المعدل", "التحصيلي", "القدرات"]

df_ex = df[experiences_cols]
df_int = df[interests_cols]
df_ex.columns = field_names
df_int.columns = field_names
df_int.loc[:, "Type"] = "Interest"
df_ex.loc[:, "Type"] = "Experience"
df_int_ex = pd.concat([df_int, df_ex])

df_ex_sum = df[experiences_cols].apply(sum)
df_int_sum = df[interests_cols].apply(sum)

d:\Softwarez\Anaconda\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
d:\Softwarez\Anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [8]:
#  Interests and experiences in one table
df_int_ex_concat = pd.concat([df_int_sum, df_ex_sum])
data = zip(*[field_names * 2, df_int_ex_concat.values / max(df_int_ex_concat.values), ["Interest"] * len(field_names) + ["Experience"] * len(field_names)])
df_int_ex_sum = pd.DataFrame(data, columns=["Field", "Score", "Type"])


#### Grades analysis
###### Distribution of all grades 

In [191]:
px.histogram(df[["القدرات", "التحصيلي"]], marginal="rug",nbins=20, width=1000, height=700)

In [188]:
px.histogram(df["المعدل"], marginal="rug", nbins=20,width=1000, height=700)

In [141]:
temp = df[["القدرات", "التحصيلي"]].copy()
temp.index = df["المعدل"]
px.scatter(temp, trendline="ols", width=1000, height=700, title="إرتباط المعدل مع القدرات والتحصيلي")


#### Interest vs. experience

In [9]:
fig = px.histogram(df_int_ex_sum, x="Field", y="Score", color="Type", barmode="group", title="Difference between Interest and Experience")
fig.update_layout(width=1500, height=500)

In [102]:
fig = px.box(df_int_ex, color="Type", boxmode="group", title="Distribution of opinion regarding Interest and Experience")
fig.update_layout(width=1500, height=500)

In [11]:
fig = px.scatter(df, x='المعدل', y="قدر عدد الجهات اللتي ردت عليك", marginal_x="histogram")
fig.update_layout(width=500, height=500)

In [12]:
fig = px.scatter(df, x='التحصيلي', y="قدر عدد الجهات اللتي ردت عليك", marginal_x="histogram")
fig.update_layout(width=500, height=500)

In [13]:
fig = px.scatter(df, x='القدرات', y="قدر عدد الجهات اللتي ردت عليك", marginal_x="histogram")
fig.update_layout(width=500, height=500)

In [14]:
mean_grades = (df['القدرات'] + df['التحصيلي'] + df['المعدل']) / 3
fig = px.scatter(df, x=mean_grades.values, y="قدر عدد الجهات اللتي ردت عليك", marginal_x="histogram")
fig.update_layout(width=500, height=500)

#### Correlation between variables

In [15]:
fig = px.imshow(df.corr())
fig.update_layout(width=1500, height=1000)

In [16]:
fig = px.bar(
    df.corr()[grade_cols].T[applying_cols], 
    barmode="group",
    title="ارتباط الأداء الدراسي مع احصائيات التدريب الميداني")
fig.update_layout(
    width=1500, 
    height=800, 
    xaxis_title="",
    yaxis_title="الأرتباط",
)

In [17]:
fig = px.treemap(
    df[df["هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب"] == "نعم"], 
    path=["التخصص", "التصنيف", "قدّر فائدة مشاريعك ككل بالنسبة للوقت المصروف عليها حسب مسيرتك المهنية"], 
    values="المعدل",
    title="المعدل -> عدد التقديمات -> عدد الردود -> عدد القبول (ازرق = عمل سابقا، احمر = لم يعمل سابقا)"
)
fig.update_layout(width=1100, height=1100)

In [18]:
categorical_cols

['هل غيرت تخصصك',
 'هل تخصصك رغبتك الأولى',
 'هل كان عندك معرفه جيدة بتخصصك',
 'تطوعت سابقا',
 'تنصح بالتطوع',
 'عمل سابقا',
 'كيف طبيعة العمل',
 'هل تنصح غيرك بعمل مماثل',
 'هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب',
 'هل كان لمشاريعك الخارجية أثر مباشر على فرصك المهنية (تدريب، توظيف، الخ)',
 'هل تنصح غيرك بالمشاريع الخارجية',
 'هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)',
 'هل لازلت تبحث عن تدريب',
 'هل قبلت في أي جهة (قبول أولي على الأقل)',
 'هل لديك سيرة ذاتية (CV)',
 'هل لديك حساب في Linkedin',
 'هل لديك حساب في Github',
 'هل أجريت مقابلة تدريب',
 'هل المقابلة تطلبت منك البرمجة',
 'تدرب سابقا',
 'هل التدريب كان في فصل صيفي',
 'تدريب بسبب علاقات',
 'وظيفة بعد التدريب',
 'الجنس',
 'التخصص',
 'الدفعة',
 'بعد أي مستوى كان التدريب',
 'كم كان الأجر بالشهر (بالآف)']

In [19]:
fig = px.sunburst(
    df[df["هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)"] == "نعم"], 
    path=["التصنيف", "قدر عدد الجهات اللتي قدمت عليها", "قدر عدد الجهات اللتي ردت عليك","قدر عدد الجهات اللتي ردت عليك بالقبول"], 
    values="قدر عدد الجهات اللتي ردت عليك بالقبول",
    color="عمل سابقا",
    title="المعدل -> عدد التقديمات -> عدد الردود -> عدد القبول (ازرق = عمل سابقا، احمر = لم يعمل سابقا)"
)
fig.update_layout(width=1100, height=1100)

In [20]:
work_vol_df = pd.pivot_table(
    df, 
    values=applying_cols, 
    index=["عمل سابقا", "تطوعت سابقا",],
    aggfunc="sum"
)

In [21]:
work_vol_df.style.background_gradient(cmap='autumn_r')